# Subsidy Efficiency Evaluation

### Database Import - SQL Query

In [ ]:
-- SELECT * 
-- FROM Fossils.fossil_fuel_subsidies 
-- -- limit 10
-- ;

In [ ]:
-- SELECT * 
-- FROM Fossils.fossil_fuel_subsidies_gdp
-- -- limit 10
-- ;

In [ ]:
-- SELECT * 
-- FROM Fossils.fossil_fuel_subsidies_per_capita
-- -- limit 10
-- ;

In [ ]:
SELECT DISTINCT 
    raw.Entity AS Entity, raw.Code AS CODE, raw.Year AS Year,
    raw."12.c.1 - Fossil-fuel subsidies (consumption and production) (billions of nominal United States dollars) - ER_FFS_CMPT_CD" AS ER_FFS_CMPT_CD, 
    gdp."12.c.1 - Fossil-fuel subsidies (consumption and production) as a proportion of total GDP (%) - ER_FFS_CMPT_GDP" AS ER_FFS_CMPT_GDP, 
    pc."12.c.1 - Fossil-fuel subsidies (consumption and production) per capita (nominal United States dollars) - ER_FFS_CMPT_PC_CD" AS ER_FFS_CMPT_PC_CD
FROM Fossils.fossil_fuel_subsidies AS raw
INNER JOIN Fossils.fossil_fuel_subsidies_gdp AS gdp
    ON raw.Entity = gdp.Entity 
    AND raw.Code = gdp.Code 
    AND raw.Year = gdp.Year
--     USING (Entity, Code, Year)    
INNER JOIN Fossils.fossil_fuel_subsidies_per_capita AS pc
    ON raw.Entity = pc.Entity 
    AND raw.Code = pc.Code 
    AND raw.Year = pc.Year
--     USING (Entity, Code, Year)    
;

* **Note**: The counts do not agree & match up between the tables, so some rows are lost due to incomplete matching during join (may do loose/full join to find the lossy data)

### Preliminary EDA

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# display(df_raw.head())
# display(df_gdp.head())
# display(df_pc.head())
display(df_join.head())